In [1]:
from pandas import DataFrame

In [2]:
import requests
from bs4 import BeautifulSoup
from bs4 import Tag
import re
import csv
import json

In [3]:
response = requests.get('http://www.foodpantries.org/')
soup = BeautifulSoup(response.content,'html.parser')
states = soup.find('div',{'class':'multicolumn'}).find_all('li')

In [4]:
states[:5]

[<li><a href="https://www.foodpantries.org/st/alaska" title="Alaska Food Pantries">Alaska</a> <em>(118)</em></li>,
 <li><a href="https://www.foodpantries.org/st/alabama" title="Alabama Food Pantries">Alabama</a> <em>(390)</em></li>,
 <li><a href="https://www.foodpantries.org/st/arkansas" title="Arkansas Food Pantries">Arkansas</a> <em>(523)</em></li>,
 <li><a href="https://www.foodpantries.org/st/arizona" title="Arizona Food Pantries">Arizona</a> <em>(287)</em></li>,
 <li><a href="https://www.foodpantries.org/st/california" title="California Food Pantries">California</a> <em>(712)</em></li>]

In [5]:
def generate_soup(url):
    response = requests.get(url)
    html = response.text

    return BeautifulSoup(html, "html.parser")

In [64]:
STR_TO_REPLACE = ['\r', '\n', '\t', '<b>', '<br>', '</b>', '@', '\\',\
                        'â', '\x80', '\x99s', '\xa0', '[read more]', '@type', '@context', '\\\\']
urls = []
for state in states:
    state_url = state.find('a')['href']
    state_name = state.find('a').text
    state_count = str(state.find('em').text)
    response = requests.get(state_url)
    html=response.text
    urls.append(state_url)
    soup = BeautifulSoup(html, "html.parser")

In [6]:
STR_TO_REPLACE = ['\r', '\n', '\t', '<b>', '<br>', '</b>', '@', '\\',\
                        'â', '\x80', '\x99s', '\xa0', '[read more]', '@type', '@context', '\\\\']

In [13]:
urls

['https://www.foodpantries.org/st/alaska',
 'https://www.foodpantries.org/st/alabama',
 'https://www.foodpantries.org/st/arkansas',
 'https://www.foodpantries.org/st/arizona',
 'https://www.foodpantries.org/st/california',
 'https://www.foodpantries.org/st/colorado',
 'https://www.foodpantries.org/st/connecticut',
 'https://www.foodpantries.org/st/district_of_columbia',
 'https://www.foodpantries.org/st/delaware',
 'https://www.foodpantries.org/st/florida',
 'https://www.foodpantries.org/st/georgia',
 'https://www.foodpantries.org/st/hawaii',
 'https://www.foodpantries.org/st/iowa',
 'https://www.foodpantries.org/st/idaho',
 'https://www.foodpantries.org/st/illinois',
 'https://www.foodpantries.org/st/indiana',
 'https://www.foodpantries.org/st/kansas',
 'https://www.foodpantries.org/st/kentucky',
 'https://www.foodpantries.org/st/louisiana',
 'https://www.foodpantries.org/st/massachusetts',
 'https://www.foodpantries.org/st/maryland',
 'https://www.foodpantries.org/st/maine',
 'https:

In [14]:
allURL = []
for START_URL in urls:
    soup = generate_soup(START_URL)
    table = soup.find('table', {'class': 'table table-striped'})
    rows = table.findAll('tr')
    for tr in rows:
        cols = tr.findAll('td')
        #print(cols)
        for element in cols:
            link1 = element.find('a').get('href')
            allURL.append(link1)

In [77]:
allURL[:13]

['https://www.foodpantries.org/ci/ak-anchor_point',
 'https://www.foodpantries.org/ci/ak-anchorage',
 'https://www.foodpantries.org/ci/ak-aniak',
 'https://www.foodpantries.org/ci/ak-bethel',
 'https://www.foodpantries.org/ci/ak-big_lake',
 'https://www.foodpantries.org/ci/ak-coffman_cove',
 'https://www.foodpantries.org/ci/ak-dillingham',
 'https://www.foodpantries.org/ci/ak-douglas',
 'https://www.foodpantries.org/ci/ak-eagle_river',
 'https://www.foodpantries.org/ci/ak-fairbanks',
 'https://www.foodpantries.org/ci/ak-fort_yukon',
 'https://www.foodpantries.org/ci/ak-haines',
 'https://www.foodpantries.org/ci/ak-homer']

In [18]:
pantries = []
start_soup = generate_soup('https://www.foodpantries.org/ci/ak-anchorage')
pantries_html = start_soup.find_all('script', type='application/ld+json')
for ph in pantries_html:
    pantry_text = ph.text
    for s in STR_TO_REPLACE:
        pantry_text = pantry_text.replace(s, '')
    pantry = json.loads(pantry_text, strict=False)
    if pantry['type'] == 'LocalBusiness':
        pantries.append(pantry)
pantries[:5]

[{'context': 'https://schema.org',
  'type': 'LocalBusiness',
  'address': {'type': 'PostalAddress',
   'streetAddress': '1121 E 10th Ave',
   'addressLocality': 'Anchorage',
   'addressRegion': 'AK',
   'postalCode': '99501'},
  'name': 'Mobile Food Pantry - Fairview Recreation Center',
  'image': 'https://www.foodpantries.org/gallery/1773_1647432713.jpg',
  'description': 'SERVICE DESCRIPTION: Provides a mobile food pantry to supplement existing food pantries throughout the service area.INTAKE: Served on a first-come-first-serve basis. Bring boxes or bags to take the food home in. Distribution sites and times change monthly. See the website for the most up to date information.ELIGIBILITY: No restrictions.Hours:The 2nd and 3rd Sat of the month2:00pm (registration at 1pm)For more information, please call. ',
  'telephone': '(907) 343-4130'},
 {'context': 'https://schema.org',
  'type': 'LocalBusiness',
  'address': {'type': 'PostalAddress',
   'streetAddress': '3600 Macinnes Street',
 

In [43]:
import json
import re

class LazyDecoder(json.JSONDecoder):
    def decode(self, s, **kwargs):
        regex_replacements = [
            (re.compile(r'([^\\])\\([^\\])'), r'\1\\\\\2'),
            (re.compile(r',(\s*])'), r'\1'),
        ]
        for regex, replacement in regex_replacements:
            s = regex.sub(replacement, s)
        return super().decode(s, **kwargs)

In [79]:
def generate_dict_per_pantry(soup):
    pantries = []
    pantries_html = soup.find_all('script', type='application/ld+json')
    for ph in pantries_html:
        pantry_text = ph.text
        for s in STR_TO_REPLACE:
            pantry_text = pantry_text.replace(s, '')
            pantry = json.loads(pantry_text, strict = False)
            if "type" in pantry.keys():
                if pantry['type'] == 'LocalBusiness':
                    pantries.append(pantry)

    return pantries

In [80]:
allPantries = []
for START_URL_FP in allURL:
    start_soup = generate_soup(START_URL_FP)
    pantries = generate_dict_per_pantry(start_soup)
    allPantries.append(pantries)

JSONDecodeError: Invalid \escape: line 12 column 35 (char 375)

In [85]:
allPantries[0]

[{'context': 'https://schema.org',
  'type': 'LocalBusiness',
  'address': {'type': 'PostalAddress',
   'streetAddress': '73358 School Street',
   'addressLocality': 'Anchor Point',
   'addressRegion': 'AK',
   'postalCode': '99556'},
  'name': 'Anchor Point Food Pantry (APFP)',
  'image': 'https://www.foodpantries.org/gallery/37163_anchor-point-food-pantry-apfp_efy.jpg',
  'description': 'Hours: Mondays4:00pm to 6:00pm At Great Land Worship Center behind Chapman School.  A hot meal is serve.  We serve 25 - 40 families per week. For more information, please call. ',
  'telephone': '(907) 756-3719'},
 {'context': 'https://schema.org',
  'type': 'LocalBusiness',
  'address': {'type': 'PostalAddress',
   'streetAddress': '73358 School Street',
   'addressLocality': 'Anchor Point',
   'addressRegion': 'AK',
   'postalCode': '99556'},
  'name': 'Anchor Point Food Pantry (APFP)',
  'image': 'https://www.foodpantries.org/gallery/37163_anchor-point-food-pantry-apfp_efy.jpg',
  'description': '

In [86]:
col_names = ["StreetAddress", "City", "State", "ZipCode", "Name", "Telephone"]

In [87]:
dic_list = []
for element in allPantries:
    for innerElement in element:
        dict1 = {"Name": innerElement["name"],
                 "StreetAddress": innerElement["address"]["streetAddress"],
                 "City": innerElement["address"]["addressLocality"],
                 "State": innerElement["address"]["addressRegion"],
                 "ZipCode": innerElement["address"]["postalCode"],
                 "Telephone": innerElement["telephone"]
                }
        dic_list.append(dict1)

In [88]:
dic_list[:5]

[{'Name': 'Anchor Point Food Pantry (APFP)',
  'StreetAddress': '73358 School Street',
  'City': 'Anchor Point',
  'State': 'AK',
  'ZipCode': '99556',
  'Telephone': '(907) 756-3719'},
 {'Name': 'Anchor Point Food Pantry (APFP)',
  'StreetAddress': '73358 School Street',
  'City': 'Anchor Point',
  'State': 'AK',
  'ZipCode': '99556',
  'Telephone': '(907) 756-3719'},
 {'Name': 'Anchor Point Food Pantry (APFP)',
  'StreetAddress': '73358 School Street',
  'City': 'Anchor Point',
  'State': 'AK',
  'ZipCode': '99556',
  'Telephone': '(907) 756-3719'},
 {'Name': 'Anchor Point Food Pantry (APFP)',
  'StreetAddress': '73358 School Street',
  'City': 'Anchor Point',
  'State': 'AK',
  'ZipCode': '99556',
  'Telephone': '(907) 756-3719'},
 {'Name': 'Anchor Point Food Pantry (APFP)',
  'StreetAddress': '73358 School Street',
  'City': 'Anchor Point',
  'State': 'AK',
  'ZipCode': '99556',
  'Telephone': '(907) 756-3719'}]

In [89]:
with open("FoodPantries.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in dic_list:
            wr.writerow(ele)

## freefood.org

In [90]:
response = requests.get('https://www.freefood.org/')
soup = BeautifulSoup(response.content,'html.parser')
states = soup.find('div',{'class':'box'}).find_all('li')

In [91]:
states[:5]

[<li><a href="https://www.freefood.org/s/connecticut">Connecticut</a> (40)</li>,
 <li><a href="https://www.freefood.org/s/rhode_island">Rhode Island</a> (131)</li>,
 <li><a href="https://www.freefood.org/s/massachusetts">Massachusetts</a> (188)</li>,
 <li><a href="https://www.freefood.org/s/vermont">Vermont</a> (25)</li>,
 <li><a href="https://www.freefood.org/s/new_hampshire">New Hampshire</a> (29)</li>]

In [92]:
urls = []
for state in states:
    state_url = state.find('a')['href']
    state_name = state.find('a').text
    response = requests.get(state_url)
    html=response.text
    urls.append(state_url)
    soup = BeautifulSoup(html, "html.parser")

In [93]:
urls[:5]

['https://www.freefood.org/s/connecticut',
 'https://www.freefood.org/s/rhode_island',
 'https://www.freefood.org/s/massachusetts',
 'https://www.freefood.org/s/vermont',
 'https://www.freefood.org/s/new_hampshire']

In [94]:
allURL = []
for START_URL in urls:
    soup = generate_soup(START_URL)
    cityURL = soup.find_all('div',{'class':'kode-text'})
    allURL.append(cityURL)
    

In [95]:
allURL[0][:5]

[<div class="kode-text">
 <a alt="Ansonia dcf offices" href="https://www.freefood.org/c/ct-ansonia" title="Ansonia DCF Offices">Ansonia (1)</a>
 </div>,
 <div class="kode-text">
 <a alt="Ashford dcf offices" href="https://www.freefood.org/c/ct-ashford" title="Ashford DCF Offices">Ashford (1)</a>
 </div>,
 <div class="kode-text">
 <a alt="Bloomfield dcf offices" href="https://www.freefood.org/c/ct-bloomfield" title="Bloomfield DCF Offices">Bloomfield (1)</a>
 </div>,
 <div class="kode-text">
 <a alt="Bridgeport dcf offices" href="https://www.freefood.org/c/ct-bridgeport" title="Bridgeport DCF Offices">Bridgeport (4)</a>
 </div>,
 <div class="kode-text">
 <a alt="Bristol dcf offices" href="https://www.freefood.org/c/ct-bristol" title="Bristol DCF Offices">Bristol (1)</a>
 </div>]

In [96]:
cityURLS = []
for cityClass in allURL:
    for outter in cityClass: 
        city_url = outter.find('a')['href']
        cityURLS.append(city_url)

In [97]:
cityURLS[:5]

['https://www.freefood.org/c/ct-ansonia',
 'https://www.freefood.org/c/ct-ashford',
 'https://www.freefood.org/c/ct-bloomfield',
 'https://www.freefood.org/c/ct-bridgeport',
 'https://www.freefood.org/c/ct-bristol']

In [98]:
url = 'https://www.freefood.org/c/ct-ansonia'
cont = []
soup = generate_soup(url)
pantry = soup.find_all('div',{'class':'text-col'})
pantry[:2]

[<div class="text-col">
 <h3><a href="https://www.freefood.org/l/ct_ansonia_christ-episcopal-church">Christ Episcopal Church</a></h3>
 <p>
                                     56 Cliff Street <br/>
                                     Ansonia, CT - 06401<br/>
                                     Phone: (203) 734-2715<br/>
 <br/>
 </p>
 <a class="btn-style-1" href="https://www.freefood.org/l/ct_ansonia_christ-episcopal-church">Details Page</a>
 </div>,
 <div class="text-col">
 <h3><a href="https://www.freefood.org/l/ct_shelton_area-congregations-together">Area Congregations Together - Spooner House</a></h3>
 <p>
                                     30 Todd Road <br/>
                                     Shelton, CT - 06484<br/>
                                     Phone: (203) 225-0453<br/>
                                     Fax: (203) 225-0459<br/>
 </p>
 <a class="btn-style-1" href="https://www.freefood.org/l/ct_shelton_area-congregations-together">Details Page</a>
 </div>]

In [99]:
content = []
for url in cityURLS:
    soup = generate_soup(url)
    pantry = soup.find_all('div',{'class':'text-col'})
    content.append(pantry)

In [100]:
content[0][:2]

[<div class="text-col">
 <h3><a href="https://www.freefood.org/l/ct_ansonia_christ-episcopal-church">Christ Episcopal Church</a></h3>
 <p>
                                     56 Cliff Street <br/>
                                     Ansonia, CT - 06401<br/>
                                     Phone: (203) 734-2715<br/>
 <br/>
 </p>
 <a class="btn-style-1" href="https://www.freefood.org/l/ct_ansonia_christ-episcopal-church">Details Page</a>
 </div>,
 <div class="text-col">
 <h3><a href="https://www.freefood.org/l/ct_shelton_area-congregations-together">Area Congregations Together - Spooner House</a></h3>
 <p>
                                     30 Todd Road <br/>
                                     Shelton, CT - 06484<br/>
                                     Phone: (203) 225-0453<br/>
                                     Fax: (203) 225-0459<br/>
 </p>
 <a class="btn-style-1" href="https://www.freefood.org/l/ct_shelton_area-congregations-together">Details Page</a>
 </div>]

In [101]:
dict_list = []
for outter in content:
    for inner in outter: 
        name = inner.find('a').text
        address = inner.find('p').text
        for s in STR_TO_REPLACE:
            address = address.replace(s, '')
        addressNew = ' '.join(address.split())
        index1 = addressNew.index(",")
        streetLine = addressNew[:index1]
        words = streetLine.split(" ")
        street = words[: len(words)-1]
        street1 = " ".join(street)
        city = words[len(words)-1]
        state = addressNew[index1 + 2: index1 + 4]
        zipcode = addressNew[index1 + 7 :index1 + 12]
        dict1 = {"name": name, "street": street1,"city": city,"state": state, "zipcode": zipcode}
        dict_list.append(dict1)

In [102]:
dict_list[:3]

[{'name': 'Christ Episcopal Church',
  'street': '56 Cliff Street',
  'city': 'Ansonia',
  'state': 'CT',
  'zipcode': '06401'},
 {'name': 'Area Congregations Together - Spooner House',
  'street': '30 Todd Road',
  'city': 'Shelton',
  'state': 'CT',
  'zipcode': '06484'},
 {'name': 'New Haven Diaper Bank',
  'street': '1440 Whalley Ave New',
  'city': 'Haven',
  'state': 'CT',
  'zipcode': '06515'}]

In [103]:
len(dict_list)

2237

In [104]:
for outter in dict_list:
    if outter["name"] == "Add a Listing":
        dict_list.remove(outter)

In [105]:
for outter in dict_list:
    if outter["name"] == "Good Neighbors":
        dict_list.remove(outter)

In [106]:
col_names = ["name", "street", "city", "state", "zipcode"]

In [107]:
dict_list[:20]

[{'name': 'Christ Episcopal Church',
  'street': '56 Cliff Street',
  'city': 'Ansonia',
  'state': 'CT',
  'zipcode': '06401'},
 {'name': 'Area Congregations Together - Spooner House',
  'street': '30 Todd Road',
  'city': 'Shelton',
  'state': 'CT',
  'zipcode': '06484'},
 {'name': 'New Haven Diaper Bank',
  'street': '1440 Whalley Ave New',
  'city': 'Haven',
  'state': 'CT',
  'zipcode': '06515'},
 {'name': 'Bethel Ame Church Food Pantry',
  'street': '255 Goffe Street New',
  'city': 'Haven',
  'state': 'CT',
  'zipcode': '06511'},
 {'name': 'Believe In Me Empowerment Corporation',
  'street': '423 Dixwell Avenue New',
  'city': 'Haven',
  'state': 'CT',
  'zipcode': '06511'},
 {'name': 'Connecticut Food Bank, Inc.',
  'street': 'P.O. Box 8686 New',
  'city': 'Haven',
  'state': 'CT',
  'zipcode': '06531'},
 {'name': 'Centro San Jose Family Center',
  'street': '290 Grand Avenue New',
  'city': 'Haven',
  'state': 'CT',
  'zipcode': '06513'},
 {'name': 'Walk of Faith Church Pantry

In [108]:
with open("FreeFood.csv", 'w') as csvFile:
        wr = csv.DictWriter(csvFile, fieldnames=col_names)
        wr.writeheader()
        for ele in dict_list:
            wr.writerow(ele)

## HOMELESS SHELTERS DIRECTORY

In [7]:
response = requests.get('https://www.homelessshelterdirectory.org/foodbanks/index.html')
soup = BeautifulSoup(response.content,'html.parser')
table = soup.find('table')
rows = table.findAll('tr')

In [8]:
def findURL(rows):
    rowList = []
    for row in rows:
        state_url = row.find_all('td')
        if len(state_url)>0:
            rowList.append(state_url)
    return rowList

In [9]:
stateUrl = findURL(rows)
len(stateUrl)

51

In [10]:
def innerUrl(urlList):
    urlList = []
    for item in stateUrl:
        url = item[0].find('a')['href']
        urlList.append(url)
    return urlList

In [11]:
urlList = innerUrl(stateUrl)
urlList[:5]

['https://www.homelessshelterdirectory.org/foodbanks/state/alaska.html',
 'https://www.homelessshelterdirectory.org/foodbanks/state/alabama.html',
 'https://www.homelessshelterdirectory.org/foodbanks/state/arkansas.html',
 'https://www.homelessshelterdirectory.org/foodbanks/state/arizona.html',
 'https://www.homelessshelterdirectory.org/foodbanks/state/california.html']

In [12]:
urls = []
response = requests.get('https://www.homelessshelterdirectory.org/foodbanks/state/alaska.html')
soup = BeautifulSoup(response.content,'html.parser')
table = soup.find('table')
rows = table.findAll('tr')
cityURL = findURL(rows)
for item in cityURL:
    url = item[0].find('a')['href']
    urls.append(url)
    


In [13]:
urls = []
for element in urlList:
    response = requests.get(element)
    soup = BeautifulSoup(response.content,'html.parser')
    table = soup.find('table')
    rows = table.findAll('tr')
    cityURL = findURL(rows)
    for item in cityURL:
        url = item[0].find('a')['href']
        urls.append(url)

In [14]:
urls[:3]

['https://www.homelessshelterdirectory.org/foodbanks/city/ak-anchorage.html',
 'https://www.homelessshelterdirectory.org/foodbanks/city/ak-aniak.html',
 'https://www.homelessshelterdirectory.org/foodbanks/city/ak-big_lake.html']

In [15]:
len(urls)

4588

In [ ]:
bankUrls = []
for element in urls:
    response = requests.get(element)
    soup = BeautifulSoup(response.content,'html.parser')
    banks = soup.find_all('div',{'class':'thumb_hover'})
    for item in banks:
        bankUrl = item.find('a')['href']
        bankUrls.append(bankUrl)

In [119]:
bankUrls[:5]

['https://www.homelessshelterdirectory.org/foodbank/ak_food-bank-of-alaska',
 'https://www.homelessshelterdirectory.org/foodbank/ak_downtown-soup-kitchen',
 'https://www.homelessshelterdirectory.org/foodbank/ak_saint-francis-house-food-pantry',
 'https://www.homelessshelterdirectory.org/foodbank/ak_emergency-food-bank-st-benedicts-outreach',
 'https://www.homelessshelterdirectory.org/foodbank/ak_first-congregational-church']

In [120]:
len(bankUrls)

19499

In [121]:
response = requests.get('https://www.homelessshelterdirectory.org/foodbank/ak_food-bank-of-alaska')
soup = BeautifulSoup(response.content,'html.parser')
title = soup.find('h1', {'entry_content'})
address = soup.find('div',{'col col_4_of_12'})
contents = address.find('p').text
for s in STR_TO_REPLACE:
     contents= contents.replace(s, '')
title
contents

'Address                                    2121 Spar Ave                                     Anchorage, AK - 99501'

In [122]:
import re
pattern = re.compile(r'(\s){2,}')

In [123]:
contents = re.sub(pattern, ', ', contents)

In [124]:
contents

'Address, 2121 Spar Ave, Anchorage, AK - 99501'

In [ ]:
addressList = []
for url in bankUrls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')
    address = soup.find('div',{'col col_4_of_12'})
    contents = address.find('p').text
    for s in STR_TO_REPLACE:
         contents= contents.replace(s, '')
    contents = re.sub(pattern, ', ', contents)
    addressList.append(contents)